In [ ]:
# langchain, upstage, ragas 설치
!pip install -qU openai langchain langchain-upstage getpass4

In [ ]:
# pinecone 설치
!pip install pinecone-client

In [ ]:
import os
import getpass
import warnings

warnings.filterwarnings("ignore")

# Upstage API key 입력할 것
try:
    if "UPSTAGE_API_KEY" not in os.environ or not os.environ["UPSTAGE_API_KEY"]:
        os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

    print("API key has been set successfully.")

except:
    print("Something wrong with your API KEY. Check your API Console again.")

In [ ]:
from pinecone import Pinecone
from langchain_upstage import UpstageEmbeddings

# Pinecone API 키 설정
try:
    if "PINECONE_API_KEY" not in os.environ or not os.environ["PINECONE_API_KEY"]:
        os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

    print("Pinecone API key has been set successfully.")
except Exception as e:
    print(f"Error while setting Pinecone API key: {e}")
    raise

# Pinecone Index 이름
index_name = "kb-docs"  # 기존에 생성된 Index 이름

# Pinecone 초기화
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index(index_name)  # 기존 Index 가져오기

In [ ]:
from langchain_upstage import UpstageDocumentParseLoader

def change_pdfs_to_docs(folder_name_, file_detail_):
  folder_path = "/content/" + folder_name_  # Colab의 루트 디렉토리 하위에 deposit 폴더 생성
  file_detail = file_detail_

  # 'deposit' 폴더의 모든 PDF 파일 탐색
  pdf_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.pdf')]

  # PDF 파일 로드 및 파일 이름 저장
  docs = []
  for pdf_file in pdf_files:
      # 파일 이름 추출
      file_name = os.path.basename(pdf_file)

      # PDF 로드
      layzer = UpstageDocumentParseLoader(
          pdf_file,
          output_format='text',  # 결과물 형태 : text
          coordinates=False  # 이미지 OCR 좌표계를 가져오지 않음
      )
      file_docs = layzer.load()

      # 각 문서에 파일 이름 메타데이터 추가
      for doc in file_docs:
          doc.metadata["file_name"] = file_name
          doc.metadata["file_detail"] = file_detail

      # 전체 문서 리스트에 추가
      docs.extend(file_docs)

  return docs
  # 로드된 결과 확인
  print(f"총 {len(docs)}개의 문서를 처리했습니다.")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def change_docs_to_splits(docs_):
  # Text Splitter 설정
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,    # 각 청크의 최대 크기
      chunk_overlap=100   # 청크 간 중첩
  )

  # Splits 생성 및 메타데이터 추가
  splits = []
  for doc in docs_:
      # 문서를 청크로 분리
      split_chunks = text_splitter.split_documents([doc])

      # 각 청크에 원래 문서의 메타데이터 추가
      for chunk in split_chunks:
          chunk.metadata.update(doc.metadata)

      # 전체 Splits에 추가
      splits.extend(split_chunks)

  return splits
  # 결과 확인
  print("Splits:", len(splits))

In [ ]:
def upload_splits_to_index(splits_):

  stats = index.describe_index_stats()
  last_record_count = stats['total_vector_count']

  # Solar-Embedding 모델 로드
  embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")

  # Splits 데이터를 벡터화하여 Pinecone에 업로드
  vectors = []
  for i, split in enumerate(splits_):
      # Solar-Embedding으로 벡터화
      vector = embedding_model.embed_query(split.page_content)

      # Pinecone에 저장할 데이터 구성
      metadata = split.metadata
      metadata["text"] = split.page_content  # 청크 내용을 메타데이터에 포함

      vectors.append({
          "id": f"split-{last_record_count + i + 1}",  # 고유 ID 설정
          "values": vector,    # 벡터 값
          "metadata": metadata # 메타데이터
      })

  # Pinecone에 데이터 업로드 (namespace 옵션 추가 가능)
  namespace = "kb_namespace"  # 원하는 namespace 설정
  index.upsert(vectors=vectors, namespace=namespace)

  uploaded_record_count = len(vectors)

  print(f"{uploaded_record_count} 개의 Splits 데이터를 '{index_name}' Index에 업로드 완료 (namespace: {namespace})!")

In [ ]:
doc1 = change_pdfs_to_docs("담보대출2", "담보대출")
split1 = change_docs_to_splits(doc1)
upload_splits_to_index(split1)